## <center><div class="alert alert-info">Car-Sales-Price-Prediction</div></center>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
car_sales = pd.read_csv("data/car-sales-extended-missing-data.csv")
car_sales.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0


Using `Pandas Profiling` for EDA purpose

In [3]:
from ydata_profiling import ProfileReport
profile = ProfileReport(car_sales, title="Profiling Report")
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
car_sales.dtypes

Make              object
Colour            object
Odometer (KM)    float64
Doors            float64
Price            float64
dtype: object

In [5]:
car_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Make           951 non-null    object 
 1   Colour         950 non-null    object 
 2   Odometer (KM)  950 non-null    float64
 3   Doors          950 non-null    float64
 4   Price          950 non-null    float64
dtypes: float64(3), object(2)
memory usage: 39.2+ KB


In [6]:
car_sales.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

Our data contained some null value, lets drop the data where Price Column is missing and fill the rest of the data

In [7]:
car_sales.dropna(subset="Price", inplace=True)

In [8]:
car_sales.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

Here we have drop the value where the price column was missing, lets fill the rest of the data

In [9]:
# Prepare the data
x = car_sales.drop("Price", axis=1) # Feature (X)
y = car_sales["Price"]

In [10]:
# Fill missing values with Scikit-Learn
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Define Columns
categorical_feature = ["Make", "Colour"]
door_feature        = ["Doors"         ]
numerical_feature   = ["Odometer (KM)" ]

# FIll categorical value with 'missing' & numerical value with mean
categorical_imputer = SimpleImputer(strategy="constant", fill_value="missing")
door_imputer        = SimpleImputer(strategy="constant", fill_value=4) # here 4 is mode. np.mode(car_sales["Doors"])
numerical_imputer   = SimpleImputer(strategy="mean")

# Create an imputer (something that fills missing data)
processing = ColumnTransformer([("categorical_process_1",categorical_imputer, categorical_feature),
                                ("door_imputer_1",door_imputer, door_feature),
                                ("numerical_imputer_1", numerical_imputer, numerical_feature)],
                              remainder="passthrough")

# Transform the data
filled_data_x = processing.fit_transform(x)
filled_data_x

array([['Honda', 'White', 4.0, 35431.0],
       ['BMW', 'Blue', 5.0, 192714.0],
       ['Honda', 'White', 4.0, 84714.0],
       ...,
       ['Nissan', 'Blue', 4.0, 66604.0],
       ['Honda', 'White', 4.0, 215883.0],
       ['Toyota', 'Blue', 4.0, 248360.0]], dtype=object)

In [11]:
car_sales.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [12]:
filled_data_x = pd.DataFrame(filled_data_x, columns=["Make", "Colour", "Doors", "Odometer (KM)"])
filled_data_x

,Make,Colour,Doors,Odometer (KM)
0,Honda,White,4.0,35431.0
1,BMW,Blue,5.0,192714.0
2,Honda,White,4.0,84714.0
3,Toyota,White,4.0,154365.0
4,Nissan,Blue,3.0,181577.0
...,...,...,...,...
945,Toyota,Black,4.0,35820.0
946,missing,White,3.0,155144.0
947,Nissan,Blue,4.0,66604.0
948,Honda,White,4.0,215883.0


In [13]:
# now convert the categorical data into numerical data
from sklearn.preprocessing import OneHotEncoder
ohe_model = OneHotEncoder()

from sklearn.compose import ColumnTransformer

# Cetegorical value
categorical_value = ["Make", "Colour", "Doors"]

# Process
process = ColumnTransformer([("process1", ohe_model, categorical_value)],
                              remainder="passthrough")

# Transformed data
transformed_filled_data_x = process.fit_transform(filled_data_x)
transformed_filled_data_x

<950x15 sparse matrix of type '<class 'numpy.float64'>'
	with 3800 stored elements in Compressed Sparse Row format>

In [14]:
# Predicting the value
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
model = RandomForestRegressor(n_estimators=100)
# Splitting the data into test and train set
x_train, x_test, y_train, y_test = train_test_split(transformed_filled_data_x, y, test_size=0.25)

In [15]:
# Finding the pattern inside the data
model.fit(x_train, y_train)

RandomForestRegressor()

In [16]:
# Score
model.score(x_test, y_test)

0.2607075779623299

In [17]:
# Predicting thr value
y_preds = model.predict(x_test)
y_preds

array([ 9899.73      ,  9296.135     , 12856.19      , 12553.14      ,
       15539.16      , 14291.65      , 18634.43      , 14152.84      ,
       17795.41      , 15037.72341017, 11598.97186508, 12328.74      ,
       13232.73375   , 15410.83      , 12757.26      , 11841.12      ,
       16792.013     , 18761.59      , 14088.58      , 24790.36      ,
       12323.8       , 18613.61      , 30425.56      , 16253.28      ,
        8366.51      , 11395.775     , 26362.        , 14014.01      ,
       18142.45831566, 17885.61116378, 25521.72      ,  7337.04      ,
       13547.64      , 18126.72      , 43568.52      , 21348.74      ,
       23553.59      , 14878.19      , 16966.13      , 10001.20666667,
       13417.7995    , 12875.08017857, 24570.02      , 20030.26628571,
       26441.75      , 18745.73      , 20406.2       , 19147.23      ,
       13817.55      ,  8552.14      , 15249.34      , 13417.7995    ,
       13363.59      , 19152.66      , 11890.52      ,  7713.11375   ,
      

In [18]:
import pickle
pickle.dump(model, open("model_car_sales_price_prediction.pkl","wb"))

### <center><div class="alert alert-info"> * * * End * * * </div></center>